In [1]:
# Instalation de conda et Gmpy2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge gmpy2

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gmpy2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    gmp-6.3.0                  |       h59595ed_1         556 KB  conda-forge
    gmpy2-2.1.5   

In [5]:

from gmpy2 import mpz, is_prime , next_prime , mpz_urandomb, is_strong_prp,random_state,mpz_random,invert,powmod, random_state
import time

### Génération de nombres premiers

In [ ]:
def get_prime(size): # size la taille en binaire
    r = random_state(time.time_ns())
    q = mpz_urandomb(r,size)
    q = q.bit_set(size-1)
    q = next_prime(q)

    return q

### Génération de clé publique et privée de Paillier

In [2]:
def get_paillier_keys(size):
    p = get_prime (int(size/2))
    while True :
       q = get_prime (int(size/2))
       if q != p:
          break
    n = p*q
    phi = (p-1)*(q-1)
    return n , [phi,max(p,q),min (q,p)]

In [6]:
# Test
pub_key, priv_key  = get_paillier_keys(512)
print("N==", pub_key)

print("phi==", priv_key[0])
print("p==", priv_key[1])
print("q==", priv_key[2])

N== 6786043428123520792234859274351193016733454864752856479354131931290022715681005687729117497073561685288817455064805869072610926371148386442334357682064399
phi== 6786043428123520792234859274351193016733454864752856479354131931290022715680840631329034391278930608378493276233227265552218015118316518958379797102533264
p== 87514217888998978205291283539819720157949259410375346209404849589014586270183
q== 77542182194106816425785626784359111420654260982535906622462634365545993260953


### Fonctions de chiffrement et de déchiffrement

In [7]:
from math import floor
def get_r(pub_key):

    seed = random_state (time.time_ns())
    r = mpz_random(seed,pub_key-1) + 1
    return r

def paillier_encrypt(message, pub_key):
    N = pub_key
    r = get_r (N)
    c = powmod(powmod(r,N,N**2) * powmod(1+N,message, N**2),1,N**2)
    return c

def paillier_decrypt(enc, priv_key, pub_key):
    phi = priv_key [0]

    N = pub_key

    enc1 = powmod(enc,phi,N**2)
    phiinv = invert (phi, N)
    enc1 = (enc1 - 1)// N * phiinv
    m = powmod(enc1,1,N)
    return m

def paillier_decrypt_CRT(enc, priv_key, pub_key):
    N = pub_key
    phi = priv_key [0]
    p = priv_key [1]
    q = priv_key [2]
    phiinv = invert (phi, N)
    qinv = invert (q**2,p**2)
    xp = powmod(enc,phi,p**2)
    xq = powmod(enc,phi,q**2)
    x = powmod (qinv*(xp - xq),1,p**2) * q **2 + xq
    m = powmod((x-1)// N * phiinv ,1 , N)
    return m

In [8]:
# Test
message = mpz(102132100)
print("clear message = ", message)
enc = paillier_encrypt(message, pub_key)
print("encrypted message = ", enc)
m = paillier_decrypt(enc, priv_key, pub_key)
print("decrypted message = ", m)

m_crt = paillier_decrypt_CRT(enc, priv_key, pub_key)
print("decrypted message by CRT = ", m_crt)

clear message =  102132100
encrypted message =  263211549866228726427531237971794199245899999673096607644775439859725919287550710642941267673292442064839022798046149721320017653633880785361532101185873963656942345132062864719678335901655464903572628272193933768017468633473175063873319766595662677906097412049706155369671556019902221873896708233598327487
decrypted message =  102132100
decrypted message by CRT =  102132100
